In [1]:
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup 

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\spras\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py38haa244fe_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                               4.10.1-py38haa244fe_0 --> 4.10.3-py38haa244fe_0




conda-4.10.3         | 3.1 MB    |            |   0% 
conda-4.10.3         | 3.1 MB    |            |   1% 
conda-4.10.3         | 3.1 MB    | 2          |   2% 
conda-4.10.3         | 3.1 MB    | 3          |   4% 
conda-4.10.3         | 3.1 MB    | 4          |   5% 
conda-4.10.3         | 3.1 MB    | 7          |   7% 
conda-4.10.3         | 3.1 MB    | 9      

## Data collection

In [3]:
blr_dataset=pd.read_csv(r'D:\Academics\blr_neighborhoods.csv')

In [4]:
blr_dataset.head()

,Neighborhood,Latitude,Longitude
0,Agram,45.813177,15.977048
1,Amruthahalli,13.066513,77.596624
2,Attur,11.663711,78.533551
3,Banaswadi,13.014162,77.651854
4,Bellandur,58.235358,26.683117


## Bangalore Coordinates

In [5]:
address = 'Bangalore, Karnataka'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore Cityare {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore Cityare 12.9791198, 77.5912997.


In [6]:
# create map of New York using latitude and longitude values
map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng,neighborhood in zip(blr_data['Latitude'], blr_data['Longitude'],blr_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

## Exploring First neighborhood in the dataset

In [7]:
CLIENT_ID = 'W41RKAIU3GVS33WBWIIB1EWQPQNTBAYN0Q3M2D2MTTIW5QR2' # your Foursquare ID
CLIENT_SECRET = '34WVI21MN3BENVUI10MFNK3GUPG44UZZOA0XV5FPTXOHQX0P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W41RKAIU3GVS33WBWIIB1EWQPQNTBAYN0Q3M2D2MTTIW5QR2
CLIENT_SECRET:34WVI21MN3BENVUI10MFNK3GUPG44UZZOA0XV5FPTXOHQX0P


In [8]:
blr_dataset.loc[0,'Neighborhood']

'Agram'

In [9]:
Agram_latitude = blr_dataset.loc[0, 'Latitude'] # neighborhood latitude value
Agram_longitude = blr_dataset.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = blr_dataset.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               Agram_latitude, 
                                                               Agram_longitude))

Latitude and longitude values of Agram are 45.813177, 15.977048.


## API call to know near by venues of first neighborhood

In [11]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={} &client_secret={} &v={} &ll={},{} &radius=500 &limit=100'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Agram_latitude, 
    Agram_longitude, 
    radius, 
    LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?&client_id=W41RKAIU3GVS33WBWIIB1EWQPQNTBAYN0Q3M2D2MTTIW5QR2 &client_secret=34WVI21MN3BENVUI10MFNK3GUPG44UZZOA0XV5FPTXOHQX0P &v=20180605 &ll=45.813177,15.977048 &radius=500 &limit=100'

In [12]:
results = requests.get(url).json()


In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-15-4243e3e1ab9c>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Amélie,Dessert Shop,45.813842,15.979011
1,Vinodol,Mediterranean Restaurant,45.811666,15.975643
2,Escape Room Enigmarium Zagreb,Escape Room,45.815129,15.976299
3,A Burger & Champagne Bar,Restaurant,45.813387,15.980151
4,Gajbica,Bistro,45.813531,15.979550
5,Capuciner Grill & Steak,Steakhouse,45.815103,15.978144
6,Corner bar,Bar,45.812930,15.979440
7,Cvjetni trg,Plaza,45.812251,15.974208
8,Express Bar,Coffee Shop,45.811987,15.979285
9,La Štruk,Pie Shop,45.814824,15.976828


In [54]:
restaurants_first_neigh = nearby_venues[nearby_venues['categories'].str.contains('Restaurant')].reset_index(drop=True)
restaurants_first_neigh.head()

,name,categories,lat,lng
0,Vinodol,Mediterranean Restaurant,45.811666,15.975643
1,A Burger & Champagne Bar,Restaurant,45.813387,15.980151
2,Heritage Croatian Food,Comfort Food Restaurant,45.810810,15.979922
3,Konoba Bracera,Mediterranean Restaurant,45.814742,15.978318
4,Brewbites Zagreb,American Restaurant,45.810962,15.976449


In [16]:
print('{} venues were returned by Foursquare for one(1st) neighborhood.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare for one(1st) neighborhood.


In [18]:
# create map of Manhattan using latitude and longitude values
first_neigh_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(first_neigh_map)  
    
first_neigh_map

## Exploring all other neighborhoods in the dataset

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
neigh_venues = getNearbyVenues(names=blr_dataset['Neighborhood'],
                                   latitudes=blr_dataset['Latitude'],
                                   longitudes=blr_dataset['Longitude']
                                  )

In [21]:
neigh_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agram,45.813177,15.977048,Amélie,45.813842,15.979011,Dessert Shop
1,Agram,45.813177,15.977048,Vinodol,45.811666,15.975643,Mediterranean Restaurant
2,Agram,45.813177,15.977048,Escape Room Enigmarium Zagreb,45.815129,15.976299,Escape Room
3,Agram,45.813177,15.977048,A Burger & Champagne Bar,45.813387,15.980151,Restaurant
4,Agram,45.813177,15.977048,Gajbica,45.813531,15.979550,Bistro


In [22]:
neigh_venues.shape

(761, 7)

## Filtering the Venue category to have only "Restaurants"

In [23]:
rest_cat = neigh_venues[neigh_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
rest_cat.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agram,45.813177,15.977048,Vinodol,45.811666,15.975643,Mediterranean Restaurant
1,Agram,45.813177,15.977048,A Burger & Champagne Bar,45.813387,15.980151,Restaurant
2,Agram,45.813177,15.977048,Heritage Croatian Food,45.810810,15.979922,Comfort Food Restaurant
3,Agram,45.813177,15.977048,Konoba Bracera,45.814742,15.978318,Mediterranean Restaurant
4,Agram,45.813177,15.977048,Brewbites Zagreb,45.810962,15.976449,American Restaurant


In [24]:
rest_cat.shape

(220, 7)

In [25]:
cat_count=neigh_venues.groupby('Neighborhood').count()
cat_count

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Achitnagar,1,1,1,1,1,1
Adugodi,4,4,4,4,4,4
Agram,100,100,100,100,100,100
Akkur,24,24,24,24,24,24
Alahalli,1,1,1,1,1,1
Amruthahalli,3,3,3,3,3,3
Anekal,3,3,3,3,3,3
Arudi,3,3,3,3,3,3
Attibele,4,4,4,4,4,4


## One hot encoding for catogorizing the values

In [27]:
# one hot encoding
blr_onehot = pd.get_dummies(rest_cat[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blr_onehot['Neighborhood'] = rest_cat['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [blr_onehot.columns[-1]] + list(blr_onehot.columns[:-1])
blr_onehot = blr_onehot[fixed_columns]

blr_onehot.head()

,Neighborhood,American Restaurant,Andhra Restaurant,Asian Restaurant,Bengali Restaurant,Chinese Restaurant,Comfort Food Restaurant,Eastern European Restaurant,Fast Food Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Maharashtrian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Agram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,Agram,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Agram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Agram,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
mean_value = blr_onehot.groupby('Neighborhood').mean().reset_index()
mean_value.head()

,Neighborhood,American Restaurant,Andhra Restaurant,Asian Restaurant,Bengali Restaurant,Chinese Restaurant,Comfort Food Restaurant,Eastern European Restaurant,Fast Food Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Maharashtrian Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Seafood Restaurant,South Indian Restaurant,Tex-Mex Restaurant,Thai Restaurant,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agram,0.047619,0.0,0.047619,0.0,0.0,0.047619,0.047619,0.047619,0.047619,0.047619,0.095238,0.0,0.0,0.0,0.0,0.238095,0.0,0.0,0.047619,0.0,0.0,0.190476,0.047619,0.000000,0.0,0.0,0.047619,0.0,0.0,0.0
1,Akkur,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,Attibele,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.666667,0.0,0.0,0.000000,0.0,0.0,0.0
3,Bagalgunte,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,Banaswadi,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.5,0.0


## Knowing the top 5  Restaurants in each neighborhood

In [31]:
top_restaurants = 5

for hood in mean_value['Neighborhood']:
    print("----"+hood+"----")
    temp =  mean_value[ mean_value['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_restaurants))
    print('\n')

----Agram----
                      venue  freq
0  Mediterranean Restaurant  0.24
1                Restaurant  0.19
2        Italian Restaurant  0.10
3          Greek Restaurant  0.05
4        Turkish Restaurant  0.05


----Akkur----
                           venue  freq
0             Chinese Restaurant   1.0
1            American Restaurant   0.0
2             Mexican Restaurant   0.0
3  Vegetarian / Vegan Restaurant   0.0
4               Udupi Restaurant   0.0


----Attibele----
                           venue  freq
0        South Indian Restaurant  0.67
1              Indian Restaurant  0.33
2            American Restaurant  0.00
3             Mexican Restaurant  0.00
4  Vegetarian / Vegan Restaurant  0.00


----Bagalgunte----
                           venue  freq
0                     Restaurant   1.0
1            American Restaurant   0.0
2              Andhra Restaurant   0.0
3  Vegetarian / Vegan Restaurant   0.0
4               Udupi Restaurant   0.0


----Banaswadi----
    

###  Most common venues  in each neighborhood

In [32]:
def return_most_common_venues(row, top_restaurants):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_restaurants]

In [33]:
top_restaurants = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_restaurants):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_rest_sorted = pd.DataFrame(columns=columns)
neighborhoods_rest_sorted['Neighborhood'] = mean_value['Neighborhood']

for ind in np.arange(mean_value.shape[0]):
    neighborhoods_rest_sorted.iloc[ind, 1:] = return_most_common_venues(mean_value.iloc[ind, :], top_restaurants)

neighborhoods_rest_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,Mediterranean Restaurant,Restaurant,Italian Restaurant,Greek Restaurant,Turkish Restaurant,Seafood Restaurant,Modern European Restaurant,Indian Restaurant,American Restaurant,Fast Food Restaurant
1,Akkur,Chinese Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
2,Attibele,South Indian Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,Seafood Restaurant
3,Bagalgunte,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
4,Banaswadi,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
5,Basaveshwaranagar,Fast Food Restaurant,Indian Restaurant,Asian Restaurant,Karnataka Restaurant,American Restaurant,South Indian Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Seafood Restaurant
6,Begur,Italian Restaurant,Korean Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant
7,Bellandur,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
8,Bidadi,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
9,Byatarayanapura,Indian Restaurant,Chinese Restaurant,Andhra Restaurant,American Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant


## Clustering to find similar neighborhoods

In [34]:
kclusters = 5

blr_rest_clustering = mean_value.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(blr_rest_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([2, 2, 2, 4, 2, 3, 2, 4, 1, 1])

In [35]:
# add clustering labels
neighborhoods_rest_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [36]:
neighborhoods_rest_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Agram,Mediterranean Restaurant,Restaurant,Italian Restaurant,Greek Restaurant,Turkish Restaurant,Seafood Restaurant,Modern European Restaurant,Indian Restaurant,American Restaurant,Fast Food Restaurant
1,2,Akkur,Chinese Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
2,2,Attibele,South Indian Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,Seafood Restaurant
3,4,Bagalgunte,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
4,2,Banaswadi,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant


In [38]:
blr_rest_clus = blr_dataset

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
blr_rest_clus =blr_rest_clus.join(neighborhoods_rest_sorted.set_index('Neighborhood'), on='Neighborhood')

blr_rest_clus.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,45.813177,15.977048,2.0,Mediterranean Restaurant,Restaurant,Italian Restaurant,Greek Restaurant,Turkish Restaurant,Seafood Restaurant,Modern European Restaurant,Indian Restaurant,American Restaurant,Fast Food Restaurant
1,Amruthahalli,13.066513,77.596624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Attur,11.663711,78.533551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Banaswadi,13.014162,77.651854,2.0,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
4,Bellandur,58.235358,26.683117,4.0,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant


## Neglecting neighborhoods which has NaN values as those neighborhoods could not find any venues

In [39]:
blr_rest_new=blr_rest_clus.dropna().reset_index(drop=True)

In [42]:
blr_rest_new.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,45.813177,15.977048,2.0,Mediterranean Restaurant,Restaurant,Italian Restaurant,Greek Restaurant,Turkish Restaurant,Seafood Restaurant,Modern European Restaurant,Indian Restaurant,American Restaurant,Fast Food Restaurant
1,Banaswadi,13.014162,77.651854,2.0,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
2,Bellandur,58.235358,26.683117,4.0,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
3,Byatarayanapura,13.062074,77.596392,1.0,Indian Restaurant,Chinese Restaurant,Andhra Restaurant,American Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant
4,Doddanekkundi,12.975720,77.694042,2.0,Indian Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant


## Changing the type of Cluster Lables column

In [43]:
blr_rest_new['Cluster Labels']=blr_rest_new['Cluster Labels'].astype(int)

In [44]:
blr_rest_new.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,45.813177,15.977048,2,Mediterranean Restaurant,Restaurant,Italian Restaurant,Greek Restaurant,Turkish Restaurant,Seafood Restaurant,Modern European Restaurant,Indian Restaurant,American Restaurant,Fast Food Restaurant
1,Banaswadi,13.014162,77.651854,2,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
2,Bellandur,58.235358,26.683117,4,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
3,Byatarayanapura,13.062074,77.596392,1,Indian Restaurant,Chinese Restaurant,Andhra Restaurant,American Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant
4,Doddanekkundi,12.975720,77.694042,2,Indian Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant


In [46]:
# create map
restaurant_cluster_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blr_rest_new['Latitude'], blr_rest_new['Longitude'], blr_rest_new['Neighborhood'], blr_rest_new['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(restaurant_cluster_map)
       
restaurant_cluster_map

## Exploring the Restaurants for each cluster 

In [47]:
blr_rest_new.loc[blr_rest_new['Cluster Labels'] == 0, blr_rest_new.columns[[0]+ list(range(3,blr_rest_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,EPIP,0,Maharashtrian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant


In [48]:
blr_rest_new.loc[blr_rest_new['Cluster Labels'] == 1, blr_rest_new.columns[[0]+ list(range(3,blr_rest_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Byatarayanapura,1,Indian Restaurant,Chinese Restaurant,Andhra Restaurant,American Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant
7,Hoodi,1,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
8,Horamavu,1,Indian Restaurant,Chinese Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant
13,Singanayakanahalli,1,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
14,Vimanapura,1,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
17,Chikkalasandra,1,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
18,Doddakallasandra,1,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
19,Gottigere,1,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
20,Jayanagar H.O,1,Indian Restaurant,Chinese Restaurant,Restaurant,Fast Food Restaurant,Mexican Restaurant,Andhra Restaurant,South Indian Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Seafood Restaurant
22,Konanakunte,1,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant


In [49]:
blr_rest_new.loc[blr_rest_new['Cluster Labels'] == 2, blr_rest_new.columns[[0]+ list(range(3,blr_rest_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agram,2,Mediterranean Restaurant,Restaurant,Italian Restaurant,Greek Restaurant,Turkish Restaurant,Seafood Restaurant,Modern European Restaurant,Indian Restaurant,American Restaurant,Fast Food Restaurant
1,Banaswadi,2,Vegetarian / Vegan Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
4,Doddanekkundi,2,Indian Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant
5,Domlur,2,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Italian Restaurant,Restaurant,Rajasthani Restaurant,Punjabi Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant
9,Indiranagar S.O (Bangalore),2,Restaurant,Italian Restaurant,Indian Restaurant,Vietnamese Restaurant,Udupi Restaurant,Tex-Mex Restaurant,Chinese Restaurant,Asian Restaurant,Turkish Restaurant,Thai Restaurant
11,Lingarajapuram,2,Andhra Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant,Restaurant
12,Sadashivanagar,2,Indian Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Seafood Restaurant,American Restaurant,Mexican Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant
15,Begur,2,Italian Restaurant,Korean Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant
16,Chickpet,2,South Indian Restaurant,Middle Eastern Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,Seafood Restaurant
23,Koramangala,2,Indian Restaurant,Italian Restaurant,Seafood Restaurant,Chinese Restaurant,Kerala Restaurant,Mexican Restaurant,Andhra Restaurant,South Indian Restaurant,Thai Restaurant,American Restaurant


In [50]:
blr_rest_new.loc[blr_rest_new['Cluster Labels'] == 3, blr_rest_new.columns[[0]+ list(range(3,blr_rest_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Kundalahalli,3,Fast Food Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant
21,Kathriguppe,3,Fast Food Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
26,Nayandahalli,3,Fast Food Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
32,Basaveshwaranagar,3,Fast Food Restaurant,Indian Restaurant,Asian Restaurant,Karnataka Restaurant,American Restaurant,South Indian Restaurant,Punjabi Restaurant,Rajasthani Restaurant,Restaurant,Seafood Restaurant
35,Msrit,3,Fast Food Restaurant,Indian Restaurant,Kerala Restaurant,American Restaurant,Middle Eastern Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant
38,Peenya Dasarahalli,3,Fast Food Restaurant,Indian Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant
46,Chikkajala,3,Fast Food Restaurant,American Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant


In [51]:
blr_rest_new.loc[blr_rest_new['Cluster Labels'] == 4, blr_rest_new.columns[[0]+ list(range(3,blr_rest_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Bellandur,4,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
27,Shanthinagar,4,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
31,Bagalgunte,4,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant
37,Nagasandra S.O (Bangalore),4,Restaurant,American Restaurant,Andhra Restaurant,Vegetarian / Vegan Restaurant,Udupi Restaurant,Turkish Restaurant,Thai Restaurant,Tex-Mex Restaurant,South Indian Restaurant,Seafood Restaurant


In [52]:
blr_rest_new.loc[blr_rest_new['Cluster Labels'] == 5, blr_rest_new.columns[[0]+ list(range(3,blr_rest_new.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
